In [51]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0




In [53]:

def custom_image_generator(image_directory, image_list, batch_size, target_size=(128, 128)):
    
    num_images = len(image_list)
    while True:  # Infinite loop to keep yielding batches
        
        for start in range(0, num_images, batch_size):
            end = min(start + batch_size, num_images)
            batch_images = []
            batch_filenames = image_list[start:end]

            for filename in batch_filenames:
                img_path = os.path.join(image_directory, filename)
           
                with Image.open(img_path) as img:
                    img = img.resize(target_size)
                    img = np.array(img)  # Convert to numpy array
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
                    img = cv2.equalizeHist(img)  # Histogram equalization
                    img = cv2.GaussianBlur(img, (5, 5), 0)  # Gaussian blur
                    img = cv2.fastNlMeansDenoising(img, None, 30, 7, 21)  # Noise reduction

                    img_array = np.expand_dims(img, axis=-1) / 255.0  # Normalize and add channel dimension
                    batch_images.append(img_array)

            yield np.array(batch_images)




In [85]:
# Load the dataset
data = pd.read_csv("/kaggle/input/ocular-disease-recognition-odir5k/full_df.csv")
data = data.dropna()
# Define directories and filename lists for left and right eye images
os.chdir("/kaggle/input/ocular-disease-recognition-odir5k/preprocessed_images")
image_directory = "/kaggle/input/ocular-disease-recognition-odir5k/preprocessed_images"
left_eye = data['Left-Fundus'].tolist()  # Ordered list for left eye
right_eye = data['Right-Fundus'].tolist()  # Ordered list for right eye
print(len(left_eye))

left_eye_images = []
right_eye_images = []
img_label = []
labels = []

for i in range(len(left_eye)):
    if os.path.isfile(left_eye[i]) and os.path.isfile(right_eye[i]):
            
            left_eye_images.append(left_eye[i])
            right_eye_images.append(right_eye[i])
            img_label.append(right_eye[i].split('.')[0].split('_')[0])
            labels.append(data[['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']].to_numpy()[i])


# Define batch size
batch_size = 32

6392


In [82]:
data[['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']].to_numpy()[0]

array([0, 0, 0, 1, 0, 0, 0, 0])

In [87]:
def combined_generator(left_generator, right_generator, labels, batch_size):
    num_images = len(labels)
    while True:
        for start in range(0, num_images, batch_size):
            end = min(start + batch_size, num_images)
            left_batch = next(left_generator)
            right_batch = next(right_generator)

            # Convert to tensors
            left_batch_tensor = tf.convert_to_tensor(left_batch, dtype=tf.float32)
            right_batch_tensor = tf.convert_to_tensor(right_batch, dtype=tf.float32)
            labels_tensor = tf.convert_to_tensor(labels[start:end], dtype=tf.float32)  # Ensure the correct dtype
            
            yield ([left_batch_tensor, right_batch_tensor], labels_tensor)



# Initialize the left and right eye generators
left_eye_generator = custom_image_generator(image_directory, left_eye_images, batch_size)
right_eye_generator = custom_image_generator(image_directory, right_eye_images, batch_size)

# Combine the two generators with labels
train_generator = combined_generator(left_eye_generator, right_eye_generator, labels, batch_size)


In [91]:
# Define the input shape for grayscale images
input_shape = (128, 128, 1)

# Function to build a model branch for each eye
def build_eye_branch(input_shape):
    base_model = EfficientNetB0(include_top=False, weights=None, input_shape=input_shape)
    x = layers.GlobalAveragePooling2D()(base_model.output)
    branch_model = models.Model(inputs=base_model.input, outputs=x)
    return branch_model

# Left eye branch
left_eye_input = layers.Input(shape=input_shape, name="left_eye_input")
left_eye_branch = build_eye_branch(input_shape)
left_eye_features = left_eye_branch(left_eye_input)

# Right eye branch
right_eye_input = layers.Input(shape=input_shape, name="right_eye_input")
right_eye_branch = build_eye_branch(input_shape)
right_eye_features = right_eye_branch(right_eye_input)

# Concatenate features from both eyes
combined_features = layers.concatenate([left_eye_features, right_eye_features])

# Fully connected layers for classification
x = layers.Dense(512, activation="relu")(combined_features)
x = layers.Dropout(0.3)(x)
output = layers.Dense(8, activation="sigmoid")(x)  # 8 diseases, sigmoid for multi-label

# Final model
model = models.Model(inputs=[left_eye_input, right_eye_input], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [98]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def create_tf_dataset(left_images, right_images, labels, batch_size):
    def generator():
        for left_image_path, right_image_path, label_batch in zip(left_images, right_images, labels):
            # Load images from file paths
            left_image = img_to_array(load_img(left_image_path, color_mode='grayscale', target_size=(128, 128)))
            right_image = img_to_array(load_img(right_image_path, color_mode='grayscale', target_size=(128, 128)))
            
            # Ensure the images are properly reshaped
            left_image = np.reshape(left_image, (128, 128, 1))
            right_image = np.reshape(right_image, (128, 128, 1))
            
            # Yield the image pair and the corresponding label
            yield ([left_image, right_image], label_batch)

    return tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            (tf.TensorSpec(shape=(128, 128, 1), dtype=tf.float32),  # Left eye images
             tf.TensorSpec(shape=(128, 128, 1), dtype=tf.float32)),  # Right eye images
            tf.TensorSpec(shape=(8,), dtype=tf.float32)  # Labels (assuming 8 classes)
        )
    ).batch(batch_size)


In [99]:
# Assuming left_eye_images, right_eye_images, labels are defined
train_dataset = create_tf_dataset(
    left_eye_images[:int(len(left_eye_images) * (1 - validation_split))],
    right_eye_images[:int(len(right_eye_images) * (1 - validation_split))],
    labels[:int(len(labels) * (1 - validation_split))],
    batch_size
)

validation_dataset = create_tf_dataset(
    left_eye_images[int(len(left_eye_images) * (1 - validation_split)):],
    right_eye_images[int(len(right_eye_images) * (1 - validation_split)):],
    labels[int(len(labels) * (1 - validation_split)):],
    batch_size
)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10
)


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was ((tf.float32, tf.float32), tf.float32), but the yielded element was ([array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]], dtype=float32), array([[[0.],
        [0.],
        [0.],
        ...,
       

In [86]:
print(f'Left images shape: {len(left_eye_images)}')
print(f'Right images shape: {len(right_eye_images)}')
print(f'Labels shape: {len(labels)}')

Left images shape: 6068
Right images shape: 6068
Labels shape: 6068
